# Create NCN (Narodowe Centrum Nauki) Awards

Creates Polish research grant awards from the NCN ranking lists. ~23.8K projects.

**Prerequisites:**
- Run `scripts/local/ncn_to_s3.py` to download and upload the data first.

**Data source:** https://dane.gov.pl/pl/dataset/2797  
**XLSX:** https://api.dane.gov.pl/media/resources/20221223/NCN_listy_rankingowe_20221220.xlsx  
**S3 location:** `s3a://openalex-ingest/awards/ncn/ncn_projects.parquet`

**NCN funder:**
- funder_id: 4320322511
- ROR: https://ror.org/04waf7p94
- DOI: 10.13039/501100004281
- display_name: "Narodowe Centrum Nauki"

**Note:** This dataset does not include start/end dates. Only competition type (edition) is available.

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.ncn_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/ncn/ncn_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~23.8K)
SELECT COUNT(*) as total_projects FROM openalex.awards.ncn_raw;

In [ ]:
%sql
-- Inspect column names
DESCRIBE openalex.awards.ncn_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.ncn_raw LIMIT 5;

## Step 2: Create NCN Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.ncn_awards
USING delta
AS
WITH
-- Get NCN funder from OpenAlex
ncn_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320322511  -- Narodowe Centrum Nauki
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(CAST(g.project_id AS STRING))))) % 9000000000 as id,

        -- Display name = project title (prefer English, fallback to Polish)
        COALESCE(NULLIF(TRIM(g.title_en), ''), g.title_pl) as display_name,

        -- No description available in this dataset
        CAST(NULL AS STRING) as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Amount in PLN
        TRY_CAST(g.amount_pln AS DOUBLE) as amount,
        'PLN' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map NCN competition types
        CASE
            WHEN UPPER(g.competition_type) IN ('PRELUDIUM', 'PRELUDIUM BIS') THEN 'fellowship'  -- PhD students
            WHEN UPPER(g.competition_type) = 'ETIUDA' THEN 'fellowship'  -- Doctoral scholarships
            WHEN UPPER(g.competition_type) = 'FUGA' THEN 'fellowship'  -- Postdoc mobility
            WHEN UPPER(g.competition_type) IN ('SONATINA', 'SONATA', 'SONATA BIS') THEN 'grant'  -- Early career
            WHEN UPPER(g.competition_type) = 'MAESTRO' THEN 'grant'  -- Senior researchers
            WHEN UPPER(g.competition_type) IN ('OPUS', 'OPUS LAP') THEN 'grant'  -- Research projects
            WHEN UPPER(g.competition_type) = 'HARMONIA' THEN 'grant'  -- International collaboration
            WHEN UPPER(g.competition_type) IN ('POLONEZ', 'POLONEZ BIS') THEN 'fellowship'  -- Incoming researchers
            WHEN UPPER(g.competition_type) IN ('BEETHOVEN', 'UNISONO') THEN 'grant'  -- Bilateral programs
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = competition type + panel
        CONCAT(g.competition_type, ' - ', COALESCE(g.panel, 'unknown')) as funder_scheme,

        -- Provenance
        'ncn_ranking_lists' as provenance,

        -- Dates - not available in this dataset
        CAST(NULL AS DATE) as start_date,
        CAST(NULL AS DATE) as end_date,
        CAST(NULL AS INT) as start_year,
        CAST(NULL AS INT) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    TRIM(CONCAT(COALESCE(g.pi_given_name, ''), ' ', COALESCE(g.pi_middle_name, ''))) as given_name,
                    g.pi_family_name as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.institution as name,
                        'Poland' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - NCN projects database
        'https://projekty.ncn.gov.pl/' as landing_page_url,

        -- No DOI for NCN grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(CAST(g.project_id AS STRING))))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.ncn_raw g
    CROSS JOIN ncn_funder f
    WHERE g.project_id IS NOT NULL
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'ncn_ranking_lists' AND priority = 32;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    32 as priority
FROM openalex.awards.ncn_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~23.8K)
SELECT COUNT(*) as total_awards FROM openalex.awards.ncn_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    lead_investigator
FROM openalex.awards.ncn_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be NCN)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.ncn_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt, ROUND(SUM(amount)/1000000, 1) as funding_millions_pln
FROM openalex.awards.ncn_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check competition type distribution (from funder_scheme)
SELECT 
    SPLIT(funder_scheme, ' - ')[0] as competition_type,
    COUNT(*) as cnt,
    ROUND(SUM(amount)/1000000, 1) as funding_millions_pln
FROM openalex.awards.ncn_awards
GROUP BY SPLIT(funder_scheme, ' - ')[0]
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_with_title,
    ROUND(SUM(amount)/1000000000, 2) as total_funding_billion_pln
FROM openalex.awards.ncn_awards;

In [ ]:
%sql
-- Check top institutions
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt,
    ROUND(SUM(amount)/1000000, 1) as funding_millions_pln
FROM openalex.awards.ncn_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check panels (discipline distribution)
SELECT 
    SPLIT(funder_scheme, ' - ')[1] as panel,
    COUNT(*) as cnt
FROM openalex.awards.ncn_awards
GROUP BY SPLIT(funder_scheme, ' - ')[1]
ORDER BY cnt DESC
LIMIT 20;